# Compare top bio and tweet identities over time

In [ ]:
# Top bio identities over time
import os
from glob import glob
import pandas as pd
from tqdm.auto import tqdm
import pdb

dfs = []

tweet_output_dirpath = '../output/tweets_bios_identities/'
bio_output_dirpath = '../output/tweets_identities/'
tweet_fpaths = sorted(glob(os.path.join(tweet_output_dirpath, '*')))

for tweet_fpath in tqdm(tweet_fpaths):
# for tweet_fpath in ['../output/tweets_bios_identities/virus_2020_1_29.jsonl']:
    fname = os.path.basename(tweet_fpath)

    # Find matching original document
    matching_bio_fpath = os.path.join(bio_output_dirpath, fname)
    
    # Load bio and tweet-identified files
    # tweets_output = pd.read_json(tweet_fpath, lines=True)
    bio_output = pd.read_json(matching_bio_fpath, lines=True)
    expanded = bio_output[['identities', 'created_at']].explode('identities')
    dfs.append(expanded)

len(dfs)

  0%|          | 0/785 [00:00<?, ?it/s]